# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv



# aws connect

from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider

from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement


In [2]:
# session.shutdown()

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/apple/pyprojects/project_data_modeling_cassandra
['/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-15-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-22-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-09-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-18-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-04-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-01-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-27-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-10-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-20-events.csv', '/Users/apple/pyprojects/project_data_modeling_cassandra/event_data/2018-11-17-events.csv', '/Users/apple/pyproject

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [16]:
df = pd.read_csv('event_datafile_new.csv')

print(df.shape)

df.head()

(6820, 11)


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Connecting to a Cluster

for aws, no need to create cluster, cluster already there in aws, just connect to it then start with creating keyspaces

In [7]:

# connect aws keyspaces (cassandra)
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('/Users/apple/sf-class2-root.crt')
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username='testuser-fill_in_user_info', password='fill_in_password')
cluster = Cluster(['cassandra.ap-east-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142) # HK region
session = cluster.connect()

# session.shutdown()

In [1]:
# Test with a read statement. (make sure tutorialkeyspace.tutorialtable is there in your region's space)

r = session.execute('SELECT * FROM tutorialkeyspace.tutorialtable;')
print(r.current_rows)

NameError: name 'session' is not defined

#### Create Keyspace

In [9]:
# Create a Keyspace
try:
    print("Creating keyspace: sparkifykeyspace")
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifykeyspace
        WITH replication = {'class': 'SingleRegionStrategy'};
    """)
    print("Creating keyspace: DONE.")

except Exception as e:
    print(e)

Creating keyspace: sparkifykeyspace
Creating keyspace: DONE.


#### Set Keyspace

In [11]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifykeyspace')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Create Table For Query #1

In [13]:
## 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Header would be: sessionId, itemInSession, artist, song, length
# Primary Key: (sessionId, itemInSession) which is the asking condition

try:
    print("Creating table: songs_by_session_and_item")
    session.execute("""
        CREATE TABLE IF NOT EXISTS sparkifykeyspace.songs_by_session_and_item (
           sessionId int,
           itemInSession int,
           artist text,
           song text,
           length float,
           PRIMARY KEY (sessionId, itemInSession)
        ) WITH COMMENT = 'Q1: artist, song title and songlength that was during sessionId 338 and itemInSession 4';
    """)
    print("Creating table: Done")
except Exception as e:
    print(e)


Creating table: songs_by_session_and_item
Creating table: Done


#### Insert Data For Query #2

In [14]:
# Insert data to table: songs_by_session_and_item. Loop each line from the combnined csv log file.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "insert into sparkifykeyspace.songs_by_session_and_item (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        # header sequence:
        # artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
        insert_data = (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
        print(insert_data)

        # session.execute(query, insert_data)

        q = SimpleStatement(
            query,
            consistency_level=ConsistencyLevel.LOCAL_QUORUM
        )

        session.execute(q, insert_data)

    print("Inserted data into table: songs_by_session_and_item.\n")


# aws need to set consistency level
# https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/

(583, 0, 'Harmonia', 'Sehr kosmisch', 655.77751)
inserted.

(583, 1, 'The Prodigy', 'The Big Gundown', 260.07465)
inserted.

(583, 2, 'Train', 'Marry Me', 205.45261)
inserted.

(597, 0, 'Sony Wonder', 'Blackbird', 218.06975)
inserted.

(602, 2, 'Van Halen', 'Best Of Both Worlds (Remastered Album Version)', 289.38404)
inserted.

(602, 3, 'Magic Sam', 'Call Me If You Need Me', 132.04853)
inserted.

(602, 4, 'Edward Sharpe & The Magnetic Zeros', 'Home', 306.31138)
inserted.

(602, 5, 'Usher featuring will.i.am', 'OMG', 395.72853)
inserted.

(602, 7, 'Helen Reddy', 'Candle On The Water', 176.50893)
inserted.

(602, 8, 'Taylor Swift', 'Our Song', 201.06404)
inserted.

(602, 9, 'Sean Paul', 'Baby Boy [feat. Beyonce]', 245.34159)
inserted.

(582, 0, 'Soundgarden', 'Black Hole Sun', 272.19546)
inserted.

(602, 10, 'The Killers', 'Human', 360.75057)
inserted.

(582, 1, 'Amy Winehouse', 'Addicted', 165.11955)
inserted.

(582, 2, 'Steve Anderson', 'Air', 265.06404)
inserted.

(582, 3, 'Rob Zombie

OperationTimedOut: errors={'198.18.1.59:9142': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=198.18.1.59:9142

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
## Add in the SELECT statement to verify the data was entered into the table
r = session.execute('SELECT * FROM sparkifykeyspace.songs_by_session_and_item WHERE sessionId = 338 AND itemInSession = 4;')
print(r.current_rows)


"""
SELECT artist, song, length FROM sparkifykeyspace.songs_by_session_and_item
WHERE sessionId = 518 and itemInSession = 9
"""

[]


'\nSELECT artist, song, length FROM sparkifykeyspace.songs_by_session_and_item\nWHERE sessionId = 518 and itemInSession = 9\n'

#### Create Table For Query #2

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Header would be: userId, sessionId, itemInSession, artist, song, firstName, lastName
# Primary Key: partition key (userId, sessionId), clustering column (itemInSession)
# Also order with the clustering column itemInSession for the requiremend in query.

try:
    print("Creating table: events_by_user_by_session")
    session.execute("""
        CREATE TABLE IF NOT EXISTS sparkifykeyspace.events_by_user_by_session (
            userId int,
            sessionId int,
            itemInSession int,
            artist text,
            song text,
            firstName text,
            lastName text,
            PRIMARY KEY ((userId, sessionId), itemInSession)
        ) WITH comment = 'q2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182'
        AND CLUSTERING ORDER BY (itemInSession ASC);
    """)
    print("Creating table: DONE")
except Exception as e:
    print(e)

#### Insert Data For Query #2

In [ ]:
# Insert data to table: events_by_user_by_session. Loop each line from the combnined csv log file.

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "insert into sparkifykeyspace.events_by_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"

        ## Assign which column element should be assigned for each column in the INSERT statement.
        # header sequence reminder:
        # 0_artist,1_firstName,2_gender,3_itemInSession,4_lastName,5_length,
        # 6_level,7_location,8_sessionId,9_song,10_userId

        insert_data = (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])

        q = SimpleStatement(
            query,
            consistency_level=ConsistencyLevel.LOCAL_QUORUM
        )

        session.execute(q, insert_data)
    print("inserted.\n")


# aws need to set consistency level
# https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/

#### Do a SELECT to verify that the data have been inserted table events_by_user_by_session

In [ ]:
r = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM sparkifykeyspace.events_by_user_by_session
    WHERE userId = 10 AND sessionId = 182;
""")

print(r.current_rows)



# r = session.execute("""
#     SELECT artist, song, firstName, lastName
#     FROM sparkifykeyspace.events_by_user_by_session;
# """)
#
# print(r.current_rows)

#### Create Table For Query #3


In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Header would be: song, firstName, lastName
# Primary Key: partition key (song, firstName, lastName)

# The query only concern user's full name according to a specifice song, so no need bring up other data.


try:
    print("Creating table: users_by_song")
    session.execute("""
        CREATE TABLE IF NOT EXISTS sparkifykeyspace.users_by_song (
            song text,
            firstName text,
            lastName text,
            PRIMARY KEY (song, firstName, lastName)
        ) WITH comment = 'q3: every user name (first and last) in my music app history who listened to the song `All Hands Against His Own`';
    """)
    print("Creating table: DONE")
except Exception as e:
    print(e)

#### Insert Data For Query #3

In [ ]:
# Insert data to table: users_by_song. Loop each line from the combnined csv log file.



with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "insert into sparkifykeyspace.users_by_song (song, firstName, lastName)"
        query = query + "values (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        ## Assign which column element should be assigned for each column in the INSERT statement.
        # header sequence reminder:
        # 0_artist,1_firstName,2_gender,3_itemInSession,4_lastName,5_length,
        # 6_level,7_location,8_sessionId,9_song,10_userId

        insert_data = (line[9], line[1], line[4])

        q = SimpleStatement(
            query,
            consistency_level=ConsistencyLevel.LOCAL_QUORUM
        )

        session.execute(q, insert_data)
    print("Inserted data into table: users_by_song.\n")


# aws need to set consistency level
# https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/

#### Do a SELECT to verify that the data have been inserted into table users_by_song

In [ ]:
r = session.execute("""
    SELECT firstName, lastName
    FROM sparkifykeyspace.users_by_song
    WHERE song = 'All Hands Against His Own';
""")

print(r.current_rows)



# r = session.execute("""
#     SELECT artist, song, firstName, lastName
#     FROM sparkifykeyspace.events_by_user_by_session;
# """)
#
# print(r.current_rows)

### Drop the tables before closing out the sessions

In [17]:
## Drop the table and keyspace before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS sparkifykeyspace.songs_by_session_and_item;")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS sparkifykeyspace.events_by_user_by_session;")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS sparkifykeyspace.users_by_song;")
except Exception as e:
    print(e)

try:
    session.execute("DROP KEYSPACE IF EXISTS sparkifykeyspace;")
except Exception as e:
    print(e)

# for aws delete keyspace, there usually a slight delay

Error from server: code=2200 [Invalid query] message="sparkifykeyspace or a table under it is currently being created or deleted."


### Close the session and cluster connection¶

In [18]:
session.shutdown()
# cluster.shutdown() # for aws no need to shutdown cluster